In [1]:
#import liblaries
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import KFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow as tf

# fix random seed for repeatability
np.random.seed(1)

#show avaliable devices for CUDA to use GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17754145622345681563
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3159411916
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8167726528245315057
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
#load ECG dataset
df = pd.read_csv('data/data_train.csv',index_col=0,header=0)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,0.86963,0.70884,0.021742,0.106310,0.122850,0.122840,0.106740,0.109360,0.111490,0.123990,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.83733,0.66053,0.073337,0.083486,0.263450,0.310620,0.341160,0.363130,0.364380,0.369560,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.77360,0.67521,0.151150,0.011496,0.043060,0.086020,0.070106,0.083915,0.089777,0.095606,...,-0.001303,0.000573,-0.000394,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.80228,0.29937,0.006945,0.111050,0.037744,0.063651,0.038509,0.053223,0.048136,0.047216,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.88790,0.59505,0.032376,0.178530,0.080367,0.113220,0.085669,0.096944,0.092185,0.096117,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
#define X/Y
X = df.iloc[:,:128].values
Y = df.iloc[:,128].values

X = X.reshape(X.shape[0],128,1)

del df
gc.collect()
X.shape

(14452, 128)

In [6]:
#condiguration of GPU 
from keras import backend as K
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.8
 
# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [7]:
#custom metric for optimization
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [11]:
from numba import cuda
from model_ECG import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
K.clear_session()


#selection of number of seeds for snapshot ensemblinging
num_seed = 10

#selection of number of folds for learning
n_folds = 5

for seed in range(1,num_seed+1):
    
    np.random.seed(seed)
    
    output_size=(1) #input shape of the data
    input_size=(128,1) #output shape of the data
    
    kf = KFold(n_folds,shuffle=True,random_state= 1) #cross-validation
    
    for _fold, (train_index, val_index) in enumerate(kf.split(X)):
        
        print("\n")
        print("FOLD {}".format(_fold))
        print("SEED {}".format(seed))
        print("\n")
        
        lr=0.1e-2 #learning rate
        epochs = 50000 # n_epochs
        batch_size = 128 #n batches
        patience = 30 #early stopping
        min_delta=0.001 #early stopping
        
        #divide data by train and validation sets
        x_train = X[train_index,:,:]
        x_val = X[val_index,:,:]
        y_train = Y[train_index]
        y_val = Y[val_index]

        #load the model
        model = ECG_rec(input_size,output_size)
        model.compile(optimizer = Adam(lr), loss = 'binary_crossentropy', metrics = [auc_roc])
        
        #early stopping
        earlystopper = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='min',min_delta=min_delta)
        checkpointer = ModelCheckpoint('models_weights/ecg_recognizer/model-ECG_rec-seed{}-fold{}.h5'.format(seed,_fold), monitor='val_loss', mode='min',verbose=1, save_best_only=True)
        #train model
        results = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, 
                            callbacks=[earlystopper, checkpointer],
                           validation_data = (x_val, y_val))
        
        #clear model in order to avoid collision
        K.clear_session()
        del model
        gc.collect()



FOLD 0
SEED 1




C:\Users\dima9\Anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`


Instructions for updating:
Please switch to tf.metrics.auc. Note that the order of the labels and predictions arguments has been switched.
Train on 11561 samples, validate on 2891 samples
Epoch 1/50000
11561/11561 [==============================] - 3s 303us/step - loss: 0.4604 - auc_roc: 0.7491 - val_loss: 0.3864 - val_auc_roc: 0.8255

Epoch 00001: val_loss improved from inf to 0.38644, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed1-fold0.h5
Epoch 2/50000
11561/11561 [==============================] - 1s 85us/step - loss: 0.3946 - auc_roc: 0.8414 - val_loss: 0.3528 - val_auc_roc: 0.8548

Epoch 00002: val_loss improved from 0.38644 to 0.35278, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed1-fold0.h5
Epoch 3/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.3639 - auc_roc: 0.8640 - val_loss: 0.3431 - val_auc_roc: 0.8713

Epoch 00003: val_loss improved from 0.35278 to 0.34307, saving model to models_weights/ecg_recognizer/mode


Epoch 00063: val_loss did not improve from 0.10058
Epoch 64/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.1260 - auc_roc: 0.9762 - val_loss: 0.1034 - val_auc_roc: 0.9764

Epoch 00064: val_loss did not improve from 0.10058
Epoch 65/50000
11561/11561 [==============================] - 1s 83us/step - loss: 0.1297 - auc_roc: 0.9765 - val_loss: 0.1009 - val_auc_roc: 0.9767

Epoch 00065: val_loss did not improve from 0.10058
Epoch 66/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.1292 - auc_roc: 0.9768 - val_loss: 0.0939 - val_auc_roc: 0.9770

Epoch 00066: val_loss improved from 0.10058 to 0.09385, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed1-fold0.h5
Epoch 67/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.1213 - auc_roc: 0.9771 - val_loss: 0.0959 - val_auc_roc: 0.9773

Epoch 00067: val_loss did not improve from 0.09385
Epoch 68/50000
11561/11561 [==============================] - 1s


Epoch 00100: val_loss did not improve from 0.07871
Epoch 101/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.1006 - auc_roc: 0.9842 - val_loss: 0.0862 - val_auc_roc: 0.9843

Epoch 00101: val_loss did not improve from 0.07871
Epoch 102/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.1012 - auc_roc: 0.9844 - val_loss: 0.0814 - val_auc_roc: 0.9844

Epoch 00102: val_loss did not improve from 0.07871
Epoch 103/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.0993 - auc_roc: 0.9845 - val_loss: 0.0784 - val_auc_roc: 0.9846

Epoch 00103: val_loss improved from 0.07871 to 0.07839, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed1-fold0.h5
Epoch 104/50000
11561/11561 [==============================] - 1s 84us/step - loss: 0.1033 - auc_roc: 0.9846 - val_loss: 0.0902 - val_auc_roc: 0.9847

Epoch 00104: val_loss did not improve from 0.07839
Epoch 105/50000
11561/11561 [==============================]

...

11562/11562 [==============================] - 1s 84us/step - loss: 0.0891 - auc_roc: 0.9877 - val_loss: 0.0697 - val_auc_roc: 0.9877

Epoch 00206: val_loss did not improve from 0.06279
Epoch 207/50000
11562/11562 [==============================] - 1s 85us/step - loss: 0.0933 - auc_roc: 0.9877 - val_loss: 0.0661 - val_auc_roc: 0.9878

Epoch 00207: val_loss did not improve from 0.06279
Epoch 208/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0906 - auc_roc: 0.9878 - val_loss: 0.0634 - val_auc_roc: 0.9878

Epoch 00208: val_loss did not improve from 0.06279
Epoch 209/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0862 - auc_roc: 0.9878 - val_loss: 0.0649 - val_auc_roc: 0.9879

Epoch 00209: val_loss did not improve from 0.06279
Epoch 210/50000
11562/11562 [==============================] - 1s 83us/step - loss: 0.0926 - auc_roc: 0.9879 - val_loss: 0.0707 - val_auc_roc: 0.9879

Epoch 00210: val_loss did not improve from 0.06279
Epoch 


Epoch 00245: val_loss did not improve from 0.05943
Epoch 246/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0863 - auc_roc: 0.9895 - val_loss: 0.0755 - val_auc_roc: 0.9895

Epoch 00246: val_loss did not improve from 0.05943
Epoch 247/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0862 - auc_roc: 0.9896 - val_loss: 0.0643 - val_auc_roc: 0.9896

Epoch 00247: val_loss did not improve from 0.05943
Epoch 248/50000
11562/11562 [==============================] - 1s 85us/step - loss: 0.0881 - auc_roc: 0.9896 - val_loss: 0.0717 - val_auc_roc: 0.9896

Epoch 00248: val_loss did not improve from 0.05943
Epoch 249/50000
11562/11562 [==============================] - 1s 85us/step - loss: 0.0918 - auc_roc: 0.9896 - val_loss: 0.0669 - val_auc_roc: 0.9896

Epoch 00249: val_loss did not improve from 0.05943
Epoch 250/50000
11562/11562 [==============================] - 1s 83us/step - loss: 0.0898 - auc_roc: 0.9897 - val_loss: 0.0678 - val_auc_ro


Epoch 00023: val_loss improved from 0.18597 to 0.18151, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed10-fold4.h5
Epoch 24/50000
11562/11562 [==============================] - 1s 85us/step - loss: 0.2026 - auc_roc: 0.9509 - val_loss: 0.1828 - val_auc_roc: 0.9516

Epoch 00024: val_loss did not improve from 0.18151
Epoch 25/50000
11562/11562 [==============================] - 1s 85us/step - loss: 0.2002 - auc_roc: 0.9521 - val_loss: 0.1796 - val_auc_roc: 0.9527

Epoch 00025: val_loss improved from 0.18151 to 0.17961, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed10-fold4.h5
Epoch 26/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.1950 - auc_roc: 0.9533 - val_loss: 0.1752 - val_auc_roc: 0.9538

Epoch 00026: val_loss improved from 0.17961 to 0.17519, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed10-fold4.h5
Epoch 27/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.1923 - auc_roc: 0

11562/11562 [==============================] - 1s 83us/step - loss: 0.1015 - auc_roc: 0.9819 - val_loss: 0.0999 - val_auc_roc: 0.9820

Epoch 00091: val_loss did not improve from 0.08986
Epoch 92/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.1009 - auc_roc: 0.9821 - val_loss: 0.0939 - val_auc_roc: 0.9822

Epoch 00092: val_loss did not improve from 0.08986
Epoch 93/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.1016 - auc_roc: 0.9823 - val_loss: 0.0967 - val_auc_roc: 0.9824

Epoch 00093: val_loss did not improve from 0.08986
Epoch 94/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.1008 - auc_roc: 0.9824 - val_loss: 0.0885 - val_auc_roc: 0.9825

Epoch 00094: val_loss improved from 0.08986 to 0.08846, saving model to models_weights/ecg_recognizer/model-ECG_rec-seed10-fold4.h5
Epoch 95/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0985 - auc_roc: 0.9826 - val_loss: 0.0910 - v

11562/11562 [==============================] - 1s 85us/step - loss: 0.0821 - auc_roc: 0.9893 - val_loss: 0.0760 - val_auc_roc: 0.9893

Epoch 00166: val_loss did not improve from 0.07072
Epoch 167/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0821 - auc_roc: 0.9893 - val_loss: 0.0737 - val_auc_roc: 0.9893

Epoch 00167: val_loss did not improve from 0.07072
Epoch 168/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0816 - auc_roc: 0.9894 - val_loss: 0.0742 - val_auc_roc: 0.9894

Epoch 00168: val_loss did not improve from 0.07072
Epoch 169/50000
11562/11562 [==============================] - 1s 84us/step - loss: 0.0802 - auc_roc: 0.9894 - val_loss: 0.0751 - val_auc_roc: 0.9894

Epoch 00169: val_loss did not improve from 0.07072
Epoch 170/50000
11562/11562 [==============================] - 1s 85us/step - loss: 0.0820 - auc_roc: 0.9895 - val_loss: 0.0770 - val_auc_roc: 0.9895

Epoch 00170: val_loss did not improve from 0.07072
Epoch 